In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("video_data.csv")
df['PIX_VALUE_DIFF'] = df['TOTAL_PIX_VALUE'].diff()
print(df)

            TIME  TOTAL_PIX_VALUE  PIX_VALUE_DIFF
0       0.033367            39532             NaN
1       0.066733            39534             2.0
2       0.100100            39538             4.0
3       0.133467            39548            10.0
4       0.166833            39539            -9.0
5       0.200200            39548             9.0
6       0.233567            39536           -12.0
7       0.266933            39536             0.0
8       0.300300            39524           -12.0
9       0.333667            39536            12.0
10      0.367033            39524           -12.0
11      0.400400            39740           216.0
12      0.433767            39753            13.0
13      0.467133            39741           -12.0
14      0.500500            39741             0.0
15      0.533867            39741             0.0
16      0.567233            37699         -2042.0
17      0.600600            37699             0.0
18      0.633967            37718            19.0


In [3]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [17]:
time_period = zero_runs(df['PIX_VALUE_DIFF'].tolist())
filtered_time_period = []
for i in range(len(time_period)):
    if time_period[i][1] - time_period[i][0] >= 6: # lag for more than 7 frames
        filtered_time_period.append(time_period[i])

print(len(filtered_time_period))

42


In [10]:
result = pd.DataFrame(columns=['START_TIME', 'END_TIME', 'TIME_PERIOD'])

for i in range(0, len(filtered_time_period)):
    start_time = df.iloc[filtered_time_period[i][0]-1]['TIME']
    end_time = df.iloc[filtered_time_period[i][1]]['TIME']
    time_diff = end_time - start_time
    result = result.append({'START_TIME': start_time, 'END_TIME': end_time, 'TIME_PERIOD': time_diff}, ignore_index=True)

print(result)

    START_TIME    END_TIME  TIME_PERIOD
0     1.034367    1.935267     0.900900
1     2.002000    2.569233     0.567233
2     3.103100    3.870533     0.767433
3     3.937267    4.404400     0.467133
4     5.805800    6.673333     0.867533
5    11.044367   11.611600     0.567233
6    13.179833   13.546867     0.367033
7    16.516500   16.783433     0.266933
8    17.450767   18.385033     0.934267
9    18.418400   18.718700     0.300300
10   32.599233   32.899533     0.300300
11   46.513133   46.746700     0.233567
12   47.647600   48.381667     0.734067
13   48.448400   48.882167     0.433767
14   54.254200   54.554500     0.300300
15   54.654600   55.021633     0.367033
16   61.027633   61.261200     0.233567
17   61.294567   61.928533     0.633967
18   61.995267   62.562500     0.567233
19   62.962900   63.863800     0.900900
20   63.930533   64.831433     0.900900
21   70.270200   70.637233     0.367033
22   70.770700   71.604867     0.834167
23   73.807067   74.407667     0.600600
